After examining how the $\tau$-ratio behaves with different lambdas, I have settled on $\lambda=0.1$ and will do simulations that way.

In [1]:
import numpy as np
import numpy.random as nr
import scipy.stats as ss
import pandas as pd

In [2]:
def fromfile_tril(*args, **kwargs):
    """Read a symmetric matrix stored as just the lower triangular part.

    For an MxM matrix, the lower triangular part takes up just N=M*(M+1)$
    instead of M^2 bytes. Reversing this formula using the quadratic equ$
    M=(sqrt(1+8N)-1)/2. This is simply a wrapper around numpy.fromfile.
    """
    X = np.fromfile(*args, **kwargs)

    N = len(X)
    M = (np.sqrt(1+8*N)-1)/2
    if int(M) != M:
        raise ValueError('Length of matrix not of form M*(M+1)/2')
    M = int(M)

    Y = np.empty((M,M), X.dtype)
    indices = np.tril_indices(M)

    Y[indices] = Y[indices[::-1]] = X

    return(Y)

In [3]:
def add_lambdaI(Sigma, l):
    return((Sigma + l*np.identity(Sigma.shape[0])) / (1+l))

In [4]:
pops = ['eas', 'eur']
files = ['../common/ld/22.{}.ld.bin'.format(pop) for pop in pops]
Sigmas = np.array([fromfile_tril(file) for file in files])

frq = pd.read_table('../common/bfile.chr/22.frq.strat', delim_whitespace=True)
mafs = [ frq.query('CLST == "{}"'.format(pop)).MAF.values for pop in pops ]

In [5]:
l = 0.1
N = 100
M = Sigmas[0].shape[0]

In [6]:
SigmaPs = [add_lambdaI(Sigma, l) for Sigma in Sigmas]

In [ ]:
Zs = [nr.multivariate_normal(np.zeros(M), SigmaP, N) for SigmaP in SigmaPs]
Zs = ss.norm(0,1).sf(Zs)
G = np.hstack([[ss.binom(2,p).isf(z) for p, z in zip(maf, Z.T)] for maf, Z in zip(mafs, Zs)])

In [11]:
from pysnptools.snpreader import SnpData, Bed

ImportError: No module named 'pysnptools'